# Building A Chatbot

In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

    * Conversational RAG : Enable a chatbot experience over an external source of data
    * Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.


In [4]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.environ["GROQ_API_KEY"]

In [5]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002725CC6D600>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002725CC6C940>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [6]:
from langchain_core.messages import HumanMessage,SystemMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Sri and I am a cheif AI engineer")
        ]
    )

AIMessage(content="Hello Sri! \n\nIt's great to meet you.  \n\nBeing a Chief AI Engineer is an exciting and challenging role. What are you currently working on that you're most passionate about? \n\n", response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 23, 'total_tokens': 69, 'completion_time': 0.083636364, 'prompt_time': 0.000782797, 'queue_time': 0.163540562, 'total_time': 0.084419161}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1c8ae103-dfdf-4f68-bda5-5baaf00b7266-0', usage_metadata={'input_tokens': 23, 'output_tokens': 46, 'total_tokens': 69})

In [7]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, My name is Sri and looking for AI Jobs"),
        AIMessage(content="Hello Sri! \n\nIt's nice to meet you.  As a chief AI engineer, you must be working on some fascinating projects.  \n\nWhat kind of AI work are you most passionate about right now?  \n\n"),
        HumanMessage(content="Hey what's is my name and what i do?")
    ]
)

AIMessage(content="You told me your name is Sri, and that you're looking for AI jobs.  \n\nIs there anything else you'd like to tell me about your experience or the types of AI roles you're interested in?  Knowing more about your background and goals can help me give you more relevant advice. 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 88, 'total_tokens': 157, 'completion_time': 0.125454545, 'prompt_time': 0.006081208, 'queue_time': 0.009224592, 'total_time': 0.131535753}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0d6ede93-4243-482d-8d7b-91f89113fbb7-0', usage_metadata={'input_tokens': 88, 'output_tokens': 69, 'total_tokens': 157})

# Message History

We can use a message History class to wrap our model and make it stateful. This wil keep track of inputs and outputs of the model, and store them in some datastore. Future interactions willthen load those messages and pass them into the chain as part of the input. Let's see how to use this!



In [ ]:
# !pip install langchain_community

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:# session_id is inherited to BaseChatMessageHistory and that will be the return type for particular function
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [10]:
config={"configurable":{"session_id":"chat1"}}

In [11]:
response=with_message_history.invoke(
    [
        HumanMessage(content="Hi My name is Sri and I am Cheif AI engineer")
     ],config=config
)

In [12]:
response.content

"It's nice to meet you, Sri!  \n\nIt's great to have a fellow AI enthusiast on board. What kind of projects are you working on?  I'm always eager to learn about new developments in the field. \n\nSince I am an AI assistant, I don't have a physical role like a Chief AI Engineer, but I can help you with tasks related to AI, such as:\n\n* **Generating creative text formats:**\n\nPoems, code, scripts, musical pieces, email, letters, etc.\n\n* **Answering your questions in a comprehensive and informative way:**\n\nEven if they are open ended, challenging, or strange.\n\n* **Summarizing factual topics:**\n\nProvide concise summaries of factual topics based on the information I have been trained on.\n\nLet me know if there's anything I can assist you with today!\n"

In [13]:
# config1={"configurable":{"session_id":"chat1"}}
with_message_history.invoke(
    [
        HumanMessage(content="What's my name")
    ],config=config
)

AIMessage(content='Your name is Sri.  \n\nI remember that from our first interaction! 😊  Is there anything else I can help you with? \n', response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 215, 'total_tokens': 246, 'completion_time': 0.056363636, 'prompt_time': 0.031318818, 'queue_time': 0.093568199, 'total_time': 0.087682454}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-726dd21e-520d-4b33-8492-db330b8a5c80-0', usage_metadata={'input_tokens': 215, 'output_tokens': 31, 'total_tokens': 246})

In [14]:
config1={"configurable":{"session_id":"chat1"}}
response=with_message_history.invoke(
    [
        HumanMessage(content="What's my name")
    ],config=config1
)
response.content

# It will remember because we are using same sessionid.

'Your name is Sri.  \n\nI remember that from our first interaction! 😊  Is there anything else I can help you with? \n'

In [15]:
config2={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [
        HumanMessage(content="What's my name")
    ],config=config2
)
response.content

# Change the session id it will not remember

"As a large language model, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to use it! 😊  \n\n"

In [16]:
response=with_message_history.invoke(
    [
        HumanMessage(content="My name is John")
    ],config=config2
)
response.content

"It's nice to meet you, John!  How can I help you today?\n"

In [17]:
response=with_message_history.invoke(
    [
        HumanMessage(content="What's my name")
    ],config=config2
)
response.content

'Your name is John.  I remember!  \n\nIs there anything else I can help you with? 😊 \n'

# Prompt templates

Prompt Templates help to turn raw user information into a format that the LLM can work with. In the case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.


In [18]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the questions to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")


    ]
)


In [19]:
chain=prompt|model

In [20]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Sri")]})

AIMessage(content="Hello Sri, it's nice to meet you! \n\nIs there anything I can help you with today? 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 30, 'total_tokens': 59, 'completion_time': 0.052727273, 'prompt_time': 0.000148869, 'queue_time': 0.014103221, 'total_time': 0.052876142}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2f4a1e5d-94aa-4033-9fd3-49245aaa33fd-0', usage_metadata={'input_tokens': 30, 'output_tokens': 29, 'total_tokens': 59})

In [21]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)


In [22]:
config={"configurable":{"session_id":"chat3"}}

In [23]:
response=with_message_history.invoke(
    [
        HumanMessage(content="Hi, My name is Sri")
    ],config=config
)

In [24]:
response

AIMessage(content="Hello Sri, it's nice to meet you! 😊\n\nIs there anything I can help you with today?  \n", response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 31, 'total_tokens': 58, 'completion_time': 0.049090909, 'prompt_time': 0.003842996, 'queue_time': 0.178439904, 'total_time': 0.052933905}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-fda6fa20-c25e-4d62-8f6e-cefb8951e4e4-0', usage_metadata={'input_tokens': 31, 'output_tokens': 27, 'total_tokens': 58})

In [25]:
# Add more complexity 

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the questions to the nest of your ability in {language}."),
        MessagesPlaceholder(variable_name="messages")


    ]
)
chain=prompt|model


In [26]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is sri")],"language":"Hindi"})
response.content

'नमस्ते श्री!  😊 \n\nमुझे खुशी है कि आपने मुझे अपना नाम बताया। आप मुझसे कुछ भी पूछ सकते हैं, मैं अपनी पूरी क्षमता के अनुसार उत्तर देने की कोशिश करूँगा। \n\n'

Let's now wrap this more complicated chain in a message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [28]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [29]:
config={"configurable":{"session_id":"chat4"}}
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi, I am Sri")],"language":"hindi"},
    config=config
)
response.content

'नमस्ते श्री! \n\nमुझे बहुत खुशी है कि आप मुझसे बात कर रहे हैं।  क्या मैं आपकी कोई मदद कर सकता हूँ? \n\n'

# Manage the Conversation History

One important concept to understand when building chatbots is how to manage conversation history. It left unmanaged, the list of messages will groe unbounded and potentially overflow the contect window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.

"Trim_messages" helper to reduce  how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system messgaes and whether to allow partial messages.

In [35]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True, # system msg is imp. we say what exactly the LM model should do.
    allow_partial=False, # i don't want partial information, I just wnat to trim.
    start_on="human"  # STart from the human coversation
)
messages=[
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="Hi!"),
    HumanMessage(content=" I like Vanilla Icecream."),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
    ]
trimmer.invoke(messages)


[SystemMessage(content="you're a good assistant"),
 HumanMessage(content=' I like Vanilla Icecream.'),
 AIMessage(content='nice'),
 HumanMessage(content='whats 2+2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

In [37]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt
    |model
)
response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What icecream do i like")],
    "language":"english"
    }
)
response.content

"As a large language model, I don't have personal preferences or memories, so I don't know what your favorite ice cream flavor is!  \n\nWhat's your favorite ice cream?  🍦  \n"

In [39]:
response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What math problem did i ask")],
    "language":"english"
    }
)
response.content

'You asked "What is 2 + 2?" 🧮 😊  \n\n\n\nIs there anything else I can help you with?\n'

In [41]:
#Let's wrap this in Message History
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [43]:
response=with_message_history.invoke(
    {
    "messages":messages + [HumanMessage(content="What's my name")],
    "language":"english"
    },
    config=config
)
response.content

"As an AI, I don't have access to any information about you, including your name.  \n\nIs there anything else I can help you with?\n"